<a href="https://colab.research.google.com/github/KuroShiroe/ml-class/blob/main/ChristopherPuglisi_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
import nltk


In [48]:
df = pd.read_csv('amazonReviewsTrain.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [49]:
df.shape

(221392, 12)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221392 entries, 0 to 221391
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   verified        221392 non-null  bool  
 1   reviewTime      221392 non-null  object
 2   reviewerID      221392 non-null  object
 3   asin            221392 non-null  object
 4   reviewerName    221358 non-null  object
 5   reviewText      221344 non-null  object
 6   summary         221344 non-null  object
 7   unixReviewTime  221392 non-null  int64 
 8   vote            33295 non-null   object
 9   style           116012 non-null  object
 10  image           3703 non-null    object
 11  rating          221392 non-null  object
dtypes: bool(1), int64(1), object(10)
memory usage: 18.8+ MB


In [51]:
df.head()

,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,rating
0,True,"05 4, 2017",A2QNLVLQHW6OOC,B00NEZNT74,Jason L.,Have 30 of these in operation great lights,Five Stars,1493856000,NaN,{'Size:': ' 4PC'},NaN,positive
1,True,"12 1, 2014",A32SHV461SUWNQ,B002Q0WTCE,Richard C.,"Good replacement. I prefer my Rovner L5, but ...",Good replacement. I prefer my Rovner L5,1417392000,NaN,NaN,NaN,positive
2,True,"12 23, 2015",A2SKHG5ULPYIMC,B0002GFP9I,BellaLatinaReina-ACP Zero-Nine-Five,My son & band teacher loves it. Really nice. B...,Really Helps With Their Practice Pad,1450828800,NaN,NaN,NaN,positive
3,True,"09 4, 2013",A2MFJ6C67AL015,B003AJVKDO,BOS116,This was a gift for a friend who is a fan of t...,Guitar picks,1378252800,NaN,{'Color:': ' Logo'},NaN,positive
4,True,"08 28, 2015",ACYCDUUE9A3T2,B004XNK7AI,jack edwards,Wish it would attach to my violin a little eas...,... to my violin a little easier but it works ...,1440720000,NaN,{'Color:': ' Black - SN-5'},NaN,positive


Most of these columns shouldn't actually matter for our prediction


In [52]:
df = df.drop(['reviewTime', 'reviewerID', 'asin', 'reviewerName', 'unixReviewTime', 'style', 'image', 'verified', 'vote'],1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,reviewText,summary,rating
0,Have 30 of these in operation great lights,Five Stars,positive
1,"Good replacement. I prefer my Rovner L5, but ...",Good replacement. I prefer my Rovner L5,positive
2,My son & band teacher loves it. Really nice. B...,Really Helps With Their Practice Pad,positive
3,This was a gift for a friend who is a fan of t...,Guitar picks,positive
4,Wish it would attach to my violin a little eas...,... to my violin a little easier but it works ...,positive


In [53]:
df.isnull().sum()[df.isnull().sum() != 0]

reviewText    48
summary       48
dtype: int64

In [54]:
df = df.dropna()
df.shape

(221297, 3)

In [55]:
df.head()

,reviewText,summary,rating
0,Have 30 of these in operation great lights,Five Stars,positive
1,"Good replacement. I prefer my Rovner L5, but ...",Good replacement. I prefer my Rovner L5,positive
2,My son & band teacher loves it. Really nice. B...,Really Helps With Their Practice Pad,positive
3,This was a gift for a friend who is a fan of t...,Guitar picks,positive
4,Wish it would attach to my violin a little eas...,... to my violin a little easier but it works ...,positive


In [56]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def data_processing(text):
    text = text.lower()
    text = re.sub(r"http\S+www\S+|https\S+", '', text, flags= re.MULTILINE)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [57]:
df.reviewText = df['reviewText'].apply(data_processing)
df.summary = df['summary'].apply(data_processing)

In [58]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [59]:
df['reviewText'] = df['reviewText'].apply(lambda x: stemming(x))
df['summary'] = df['summary'].apply(lambda x: stemming(x))

In [60]:
df['text'] = df['reviewText'] + " " + df['summary']
df = df.drop(['reviewText', 'summary'],1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,rating,text
0,positive,30 operation great lights five stars
1,positive,good replacement prefer rovner l5 theres nothi...
2,positive,son band teacher loves really nice bought pad ...
3,positive,gift friend fan beatles happy gift guitar picks
4,positive,wish would attach violin little easier works g...


In [61]:
from sklearn.model_selection import train_test_split

X = df.drop(['rating'],1)
Y = df['rating']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df['text'])


In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12345)

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, Y_train)
logreg_pred = logreg.predict(X_test)
logreg_acc = accuracy_score(logreg_pred, Y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

Test accuracy: 91.04%


In [65]:
test = pd.read_csv('amazonReviewsTest.csv')

In [66]:
test.head()

,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,False,"08 17, 2016",AMKHFIJJ81LYJ,B00EA1U1X6,Frank P.,I am a (very amateur) EDM and Hip Hop Producer...,Extremely Happy.,1471392000,NaN,"{'style:': ' 5"" Monitor Speaker'}",NaN
1,True,"08 22, 2015",A3SZ045Z7CP56,B0002GNUYU,David A. Caufield,These are pretty beefy strings even for black ...,"once again, Rotosound strings fail to impress ...",1440201600,NaN,NaN,NaN
2,True,"07 30, 2014",AZMSVSJFACBPB,B0006LOBA8,Neurofizz,"I've played every new ""improved"" string out th...",and keep coming home to the girls that really ...,1406678400,NaN,NaN,NaN
3,True,"03 9, 2012",A39SRYB2M0Z5FG,B000Y30NAA,s.reeths,"I thought I would give this pedal a try, becau...",save up for a better pedal,1331251200,NaN,{'Style:': ' CHORUS'},NaN
4,True,"02 3, 2013",A35A62ZXXKXHIW,B002N4ZBE6,Marine Docl,Been wanting to try one for over a year. Good...,Big Six,1359849600,NaN,NaN,NaN


First apply the same preprocessing to our test dataframe

In [67]:
test = test.drop(['reviewTime', 'reviewerID', 'asin', 'reviewerName', 'unixReviewTime', 'style', 'image', 'verified', 'vote'],1)
test['summary'] = test['summary'].fillna("")
test.reviewText = test['reviewText'].apply(data_processing)
test.summary = test['summary'].apply(data_processing)
test['reviewText'] = test['reviewText'].apply(lambda x: stemming(x))
test['summary'] = test['summary'].apply(lambda x: stemming(x))
test['text'] = test['reviewText'] + " " + test['summary']
test = test.drop(['reviewText', 'summary'],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [68]:
test.head()

,text
0,amateur edm hip hop producer tight budget exce...
1,pretty beefy strings even black nylons im used...
2,ive played every new improved string keep comi...
3,thought would give pedal try inexpensive usual...
4,wanting try one year good quality sound like s...


In [69]:
test = cv.transform(test['text'])

In [70]:
logreg_test_prediction = logreg.predict(test)

In [71]:
print(logreg_test_prediction)

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']


because it seems to mostly be positive we'll quickly check if the AI didn't underfit and decide to make everything positive.

In [72]:
values, counts = np.unique(logreg_test_prediction, return_counts=True)
print(values)
print(counts)

['negative' 'neutral' 'positive']
[ 561  515 8924]


In [73]:
with open("amazonReviewsPredictions.txt", "w") as txt_file:
    for line in logreg_test_prediction:
        txt_file.write("".join(line) + "\n")

In [74]:
def test():
  i = 0
  answers = ['negative', 'neutral', 'positive']
  with open('amazonReviewsPredictions.txt', 'r') as outfile:
    lines = outfile.readlines()
    for line in lines:
        if line.strip() not in answers:
          return("invalid entry line %i" % (i))
        i += 1
    if i != 10000:
      return ("incorrect number of lines")
    else:
      return ("file looks ok")
test()

'file looks ok'